In [1]:
import plotly.express as px
import numpy as np
import random
from scipy import optimize

import scipy
import mlrose_hiive

In [3]:
import time

time.time()

1665861881.5817752

In [244]:
import pandas as pd
from sklearn.tree import DecisionTreeClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier

from sklearn.preprocessing import MinMaxScaler, MaxAbsScaler, StandardScaler, QuantileTransformer

from sklearn.model_selection import StratifiedShuffleSplit, StratifiedKFold, GridSearchCV, validation_curve, learning_curve
from sklearn.metrics import (RocCurveDisplay, precision_recall_curve, roc_curve, det_curve, roc_auc_score, brier_score_loss,
                            confusion_matrix, ConfusionMatrixDisplay, classification_report, accuracy_score,
                            make_scorer)

import plotly.express as px
import matplotlib.pyplot as plt
import numpy as np

df = pd.read_csv("train.csv")
df['Label'] = df['Label'].map({'subjective': 1, 'objective': 0})
frequency_columns = ['CC', 'CD', 'DT', 'EX', 'FW', 'INs', 'JJ',
       'JJR', 'JJS', 'LS', 'MD', 'NN', 'NNP', 'NNPS', 'NNS', 'PDT', 'POS',
       'PRP', 'PRP$', 'RB', 'RBR', 'RBS', 'RP', 'SYM', 'TOs', 'UH', 'VB',
       'VBD', 'VBG', 'VBN', 'VBP', 'VBZ', 'WDT', 'WP', 'WP$', 'WRB',
       'baseform', 'Quotes', 'questionmarks', 'exclamationmarks',
       'fullstops', 'commas', 'semicolon', 'colon', 'ellipsis',
       'pronouns1st', 'pronouns2nd', 'pronouns3rd', 'compsupadjadv',
       'past', 'imperative', 'present3rd', 'present1st2nd','semanticobjscore','semanticsubjscore']

sss = StratifiedShuffleSplit(test_size=0.2, random_state=42, n_splits=2)
for train_idx, test_idx in sss.split(df, df['Label']):
    train_df = df.iloc[train_idx].reset_index(drop=True)
    hold_out_df = df.iloc[test_idx].reset_index(drop=True)

frequency_norm_cols = [col + '_norm' for col in frequency_columns]
train_cols = frequency_norm_cols + ['totalWordsCount_norm','sentence1st','sentencelast']

for col in frequency_columns:
    train_df[col + '_norm'] = train_df[col] / train_df['totalWordsCount']
    hold_out_df[col + '_norm'] = hold_out_df[col] / hold_out_df['totalWordsCount']
    
ss_word_count = StandardScaler()

train_df['totalWordsCount_norm'] = ss_word_count.fit_transform(train_df['totalWordsCount'].values.reshape(-1, 1))
hold_out_df['totalWordsCount_norm'] = ss_word_count.transform(hold_out_df['totalWordsCount'].values.reshape(-1, 1))

exclude = [e + '_norm' for e in ['JJS','NNP','PRP','TOs','VBD','VBG','WP','WRB','exclamationmarks','fullstops','commas','semicolon','colon','ellipsis','EX','FW','INs']]
train_cols = [e for e in train_cols if e not in exclude]

train_df

,TextID,URL,Label,totalWordsCount,semanticobjscore,semanticsubjscore,CC,CD,DT,EX,...,pronouns2nd_norm,pronouns3rd_norm,compsupadjadv_norm,past_norm,imperative_norm,present3rd_norm,present1st2nd_norm,semanticobjscore_norm,semanticsubjscore_norm,totalWordsCount_norm
0,Text0165,http://msn.foxsports.com/horseracing/story/Twi...,0,692,36,24,7,49,2,21,...,0.001445,0.039017,0.011561,0.030347,0.005780,0.052023,0.031792,0.052023,0.034682,0.018656
1,Text0290,http://www.foxsportsohio.com/01/22/13/Kyrie-dr...,0,651,52,10,14,58,0,0,...,0.006144,0.021505,0.013825,0.035330,0.006144,0.023041,0.021505,0.079877,0.015361,-0.061847
2,Text0554,http://www.nba.com/pistons/features/truebluepi...,0,1294,87,11,79,123,1,12,...,0.000773,0.031685,0.004637,0.056414,0.003864,0.023184,0.018547,0.067233,0.008501,1.200675
3,Text0373,http://msn.foxsports.com/nascar/story/Big-chan...,0,76,2,2,1,7,0,0,...,0.000000,0.013158,0.000000,0.000000,0.013158,0.065789,0.013158,0.026316,0.026316,-1.190852
4,Text0639,http://www.wnba.com/news/cares_week_050812.html,0,1019,38,22,16,93,0,18,...,0.000000,0.004907,0.002944,0.001963,0.009814,0.014720,0.003925,0.037291,0.021590,0.660716
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
795,Text0442,http://www.foxsportssouthwest.com/01/10/13/Num...,1,1090,68,25,14,123,3,10,...,0.002752,0.035780,0.008257,0.020183,0.009174,0.023853,0.021101,0.062385,0.022936,0.800123
796,Text0878,http://www.grantland.com/story/_/id/8683881/ma...,1,2442,132,36,69,244,3,34,...,0.003686,0.041769,0.011876,0.039722,0.005324,0.033989,0.012285,0.054054,0.014742,3.454757
797,Text0508,http://msn.foxsports.com/boxing/story/floyd-ma...,0,981,60,14,20,67,0,7,...,0.004077,0.044852,0.013252,0.030581,0.001019,0.036697,0.020387,0.061162,0.014271,0.586103
798,Text0697,http://msn.foxsports.com/golf/story/tiger-wood...,1,1019,49,26,24,91,4,11,...,0.003925,0.037291,0.008832,0.040236,0.004907,0.040236,0.012758,0.048086,0.025515,0.660716


In [285]:
gds = ['gradient_descent']
gd_curves = []
for gd in gds:
    start_time = time.time()
    nn = mlrose_hiive.neural.NeuralNetwork(hidden_nodes=[5, 5],
                     activation='tanh',
                     algorithm=gd,
                     max_iters=1000,
                     bias=True,
                     is_classifier=True,
                     learning_rate=0.001,
                     early_stopping=False,
                     clip_max=3,
                     restarts=res,
                     schedule=mlrose_hiive.algorithms.decay.GeomDecay(),
                     pop_size=200,
                     mutation_prob=0.1,
                     max_attempts=10,
                     random_state=None,
                     curve=True)
    nn.fit(train_df[train_cols], train_df['Label'])
    end_time = time.time()
    gd_curves.append((gd, nn.fitness_curve, accuracy_score(hold_out_df['Label'], nn.predict(hold_out_df[train_cols])), end_time-start_time))

In [277]:
restarts = [0, 10, 100, 1000]
restart_curves = []
for res in restarts:
    print(res)
    start_time = time.time()
    nn = mlrose_hiive.neural.NeuralNetwork(hidden_nodes=[5, 5],
                     activation='tanh',
                     algorithm='random_hill_climb',
                     max_iters=1000,
                     bias=True,
                     is_classifier=True,
                     learning_rate=0.001,
                     early_stopping=False,
                     clip_max=3,
                     restarts=res,
                     schedule=mlrose_hiive.algorithms.decay.GeomDecay(),
                     pop_size=200,
                     mutation_prob=0.1,
                     max_attempts=10,
                     random_state=None,
                     curve=True)
    nn.fit(train_df[train_cols], train_df['Label'])
    end_time = time.time()
    restart_curves.append((res, nn.fitness_curve, accuracy_score(hold_out_df['Label'], nn.predict(hold_out_df[train_cols])), end_time-start_time))

0
10
100
1000


In [317]:
schedules = [('exponential', mlrose_hiive.algorithms.decay.ExpDecay(init_temp=1.0, exp_const=0.005, min_temp=0.001))]
#('geometric', mlrose_hiive.algorithms.decay.GeomDecay(init_temp=1.0, decay=0.99, min_temp=0.001)),
#          ('arithmetic', mlrose_hiive.algorithms.decay.ArithDecay(init_temp=1.0, decay=0.99, min_temp=0.001)),
#          ('exponential', mlrose_hiive.algorithms.decay.ExpDecay(init_temp=1.0, exp_const=0.005, min_temp=0.001))
schedule_curves = []
for sched in schedules:
    print(res)
    start_time = time.time()
    nn = mlrose_hiive.neural.NeuralNetwork(hidden_nodes=[5, 5],
                     activation='tanh',
                     algorithm='simulated_annealing',
                     max_iters=1000,
                     bias=True,
                     is_classifier=True,
                     learning_rate=0.001,
                     early_stopping=False,
                     clip_max=3,
                     restarts=res,
                     schedule=sched[1],
                     pop_size=200,
                     mutation_prob=0.1,
                     max_attempts=10,
                     random_state=None,
                     curve=True)
    nn.fit(train_df[train_cols], train_df['Label'])
    end_time = time.time()
    schedule_curves.append((sched[0], nn.fitness_curve, accuracy_score(hold_out_df['Label'], nn.predict(hold_out_df[train_cols])), end_time-start_time))

1000


In [281]:
mut_probs = [0.1]#, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]
mut_curves = []
for prob in mut_probs:
    print(res)
    start_time = time.time()
    nn = mlrose_hiive.neural.NeuralNetwork(hidden_nodes=[5, 5],
                     activation='tanh',
                     algorithm='genetic_alg',
                     max_iters=1000,
                     bias=True,
                     is_classifier=True,
                     learning_rate=0.001,
                     early_stopping=False,
                     clip_max=3,
                     pop_size=train_df.shape[0],
                     mutation_prob=prob,
                     max_attempts=10,
                     random_state=None,
                     curve=True)
    nn.fit(train_df[train_cols], train_df['Label'])
    end_time = time.time()
    mut_curves.append((prob, nn.fitness_curve, accuracy_score(hold_out_df['Label'], nn.predict(hold_out_df[train_cols])), end_time-start_time))

1000


In [316]:
loss_df = pd.concat([
    pd.DataFrame(
        [(i, 'RHC', val) for i, val in enumerate(restart_curves[0][1][:, 0])],
        columns=['Iterations', 'Algorithm', 'Loss']
    ),
    pd.DataFrame(
        [(i, 'SA', val) for i, val in enumerate(schedule_curves[0][1][:, 0])],
        columns=['Iterations', 'Algorithm', 'Loss']
    ),
    pd.DataFrame(
        [(i, 'GA', val) for i, val in enumerate(mut_curves[0][1][:, 0])],
        columns=['Iterations', 'Algorithm', 'Loss']
    ),
    pd.DataFrame(
        [(i, 'GD', val) for i, val in enumerate(gd_curves[0][1]*-1)],
        columns=['Iterations', 'Algorithm', 'Loss']
    )
], axis=0).reset_index(drop=True)

import plotly.express as px

px.line(loss_df, x='Iterations', y='Loss', color='Algorithm', title='Loss per Iteration using different optimizers')